In [1]:
%load_ext autoreload 
%autoreload 1
import os
# os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'
from set_env import *
set_env2()
from snorkel import SnorkelSession
import snorkel.contrib.brat as brt
from snorkel.parsers import StanfordCoreNLPServer

from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

Using postgres 2


In [2]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [3]:
brat = brt.Brat(session)

In [4]:
# Only run on documents which have annotations

# from snorkel.parser import TextDocPreprocessor, CorpusParser
# import multiprocessing
# doc_preprocessor = TextDocPreprocessor("brat_data/test_labeled_docs/*.txt", encoding="utf-8")
# corpus_parser = CorpusParser()
# corpus_parser.apply(doc_preprocessor, parralelism=multiprocessing.cpu_count())

In [5]:
input_dir = "brat_data/try_2/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

PMC2295252 {'char_start': 6524, 'span': [u'ca'], 'sent_id': 116, 'mention': u'ca. six extra rosette leaves,', 'char_end': 6553, 'idx_span': (4, 9)} Couldn't find sentence


In [6]:
pheno_spans = brat.explore()
print(len(pheno_spans))
print len([pheno_span for pheno_span in pheno_spans if len(pheno_span) > 1])
for p in pheno_spans:
    print p

680
126
[TemporarySpan("chromosome fragmentation in Atmnd1 was suppressed by loss of Atspo11-1", sentence=459551, chars=[62,131], words=[9,18])]
[TemporarySpan("did not affect normal vegetative development but caused fragmentation and missegregation of chromosomes in male and female meiosis, formation of inviable gametes, and sterility", sentence=459549, chars=[15,190], words=[3,28])]
[TemporarySpan("sensitive to salt-induced osmotic stress", sentence=449905, chars=[15,54], words=[4,8])]
[TemporarySpan("root growth was reduced in comparison with wild type on 50 and 100 mM NaCl", sentence=449899, chars=[108,181], words=[24,38])]
[TemporarySpan("sensitive to other monovalent salts, such as KCl and LiCl, and to mannitol", sentence=449903, chars=[26,99], words=[5,19])]
[TemporarySpan("more sensitive to salt stress", sentence=449894, chars=[37,65], words=[8,12])]
[TemporarySpan("modestly salt sensitive", sentence=450019, chars=[276,298], words=[50,52])]
[TemporarySpan("Salt stress sensitivi

In [7]:
from snorkel.models import Document, Sentence, Span

In [8]:
PhenoPairComplex = candidate_subclass('ComplexPhenotypes', ['descriptor', 'entity'])

In [40]:
import collections
from sqlalchemy import and_
from snorkel.models import StableLabel, GoldLabel, GoldLabelKey

jaccard_cutoff = 0.3

# ak = session.query(GoldLabelKey).filter(GoldLabelKey.name == 'gold_complex').first()
# print "label count", session.query(GoldLabel).filter(GoldLabel.key == ak).count()
# if ak is None:
#     ak = GoldLabelKey(name='gold_complex')
#     session.add(ak)
#     session.commit()
# else:
#     # Clear the labels (oh boy)
#     session.query(GoldLabel).filter(GoldLabel.key == ak).delete()
#     print "label count", session.query(GoldLabel).filter(GoldLabel.key == ak).count()

candidates = session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 3).all()
print(len(candidates), "candidates")
sentence_to_pheno = collections.defaultdict(list)

for pheno_span in pheno_spans:
    # pheno_span is a list of fragments, where each fragment is a temporary span
    if pheno_span:
        sent_id = pheno_span[0].sentence.id
        sentence_to_pheno[sent_id].append(pheno_span)

num = 0
num_matches = 0
num_sentences = 0
num_gold_phenos = 0
num_zero_matches = 0
avg = 0
num_skipped = 0
num_skipped_gap = 0

labeled = {}
j=0
for sentence_id, gold_phenos in sentence_to_pheno.iteritems():
    if j%1000==0:print num
    j+=1
    sentence_match = False
    results = session.query(Span, PhenoPairComplex).filter(and_(Span.sentence_id == sentence_id, PhenoPairComplex.descriptor_id == Span.id)).all()
    if (len(results) == 0):
        num_zero_matches += 1
#         print "sentence", sentence_id
#         for gold_pheno in gold_phenos:
#             print(gold_pheno.get_span())
#         print
        
    for gold_pheno_fragments in gold_phenos:
        matched = False
        for span, cand_pheno in results:
            # Build word indice set for each of the gold phenotype, and the candiate phenotype
            cand_words = set()
            gold_words = set()
            # combine both entities (modifier and attribute) into one "phenotype"
            [cand_words.update(xrange(span.get_word_start(), span.get_word_end()+1)) for span in cand_pheno.get_contexts()]
             
            for gold_pheno in gold_pheno_fragments:
                gold_words.update(xrange(gold_pheno.get_word_start(), gold_pheno.get_word_end()+1))
            
            # Compute distance for each candidate word
            distances = []
            for word_idx in cand_words:
                if word_idx in gold_words:
                    distance = 0
                else:
                    frag_distances = []
                    for gold_pheno in gold_pheno_fragments:
                        frag_min = gold_pheno.get_word_start()
                        frag_max = gold_pheno.get_word_end()
                        if word_idx < frag_min:
                            frag_distances.append(frag_min - word_idx)
                        elif word_idx > frag_max:
                            frag_distances.append(word_idx - frag_max)
                    
                    word_distance = min(frag_distances)
                    distances.append(word_distance)
            
            total_distance = sum([word_distance ** 2 for word_distance in distances])
            
            
            intersect = gold_words.intersection(cand_words)
            jaccard_score = float(len(intersect)) / len(cand_words.union(gold_words))
            
            
            val = -1
                
            if jaccard_score > jaccard_cutoff:
                cont = False
                if max(gold_words) - min(gold_words) <= len(gold_words)+4 and max(cand_words)-min(cand_words) > len(cand_words)+4:
                    num_skipped_gap += 1
                    cont = True
                if total_distance / len(gold_words) > 500:
                    num_skipped += 1
                    cont = True
                    
                if cont:
                    continue
#                 print "dist", total_distance, "jacard", jaccard_score
                matched = True
                sentence_match = True
                avg += jaccard_score
                num += 1
                val = 1
            
#             label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == cand_pheno).first()
#             if label is None:
#                 session.add(GoldLabel(candidate=cand_pheno, key=ak, value=val))
#             elif label.value == -1 and val == 1:
#                 label.value = val
            
        num_gold_phenos += 1
        if matched:
            num_matches += 1
    if sentence_match:
        num_sentences += 1
        
session.commit()

print("num skipped", num_skipped)
print("num_skipped_gap", num_skipped_gap)
print("avg", avg / num)
print("num matched", num_matches)
print("num candidates labeled +1", num)
recall = float(num_matches) / num_gold_phenos
print("num gold phenos", num_gold_phenos)
num_sentences_tagged = len(sentence_to_pheno.keys())
print("num_sentences_tagged by tanya", num_sentences_tagged)
print("sentences", float(num_sentences) / num_sentences_tagged)
print("num sents missed", float(num_zero_matches) / num_sentences_tagged)
print("recall", recall)

(49042, 'candidates')
0
('num skipped', 45)
('num_skipped_gap', 615)
('avg', 0.5934767030107155)
('num matched', 524)
('num candidates labeled +1', 1470)
('num gold phenos', 680)
('num_sentences_tagged by tanya', 611)
('sentences', 0.7937806873977087)
('num sents missed', 0.07692307692307693)
('recall', 0.7705882352941177)


In [10]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):

print len(candidates)
for i,c in enumerate(candidates):
    #label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == c).first()
    #if label is not None and label.value == 1: print c, label
    print type(c).type, c
    if i > 5:
        break
print
    
sents = list(set([pheno_span[0].sentence.id for pheno_span in pheno_spans if pheno_span]))
candidates = [c for c in candidates if c[0].sentence.id in sents]
    
# ProteinComponent relations
#candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()

49042
ComplexPhenotypes.type ComplexPhenotypes(Span("average", sentence=421357, chars=[3,9], words=[1,1]), Span("expected based on the data of Schwab etal", sentence=421357, chars=[83,123], words=[17,24]))
ComplexPhenotypes.type ComplexPhenotypes(Span("Immature", sentence=412035, chars=[4,11], words=[3,3]), Span("Immature leaf", sentence=412035, chars=[4,16], words=[3,4]))
ComplexPhenotypes.type ComplexPhenotypes(Span("activity", sentence=409904, chars=[83,90], words=[16,16]), Span("inflorescence", sentence=409904, chars=[53,65], words=[11,11]))
ComplexPhenotypes.type ComplexPhenotypes(Span("inflorescence", sentence=409904, chars=[53,65], words=[11,11]), Span("inflorescence", sentence=409904, chars=[53,65], words=[11,11]))
ComplexPhenotypes.type ComplexPhenotypes(Span("was further confirmed by the", sentence=425979, chars=[154,181], words=[26,30]), Span("other WD40 repeat proteins", sentence=425979, chars=[28,53], words=[4,7]))
ComplexPhenotypes.type ComplexPhenotypes(Span("was further

### this is how we can selectively only look at the labeled sentences for training/lf functions

In [11]:
sents = set([pheno_span[0].sentence.id for pheno_span in pheno_spans if pheno_span])
print len(sents)
c = [p for p in session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 3).all() if p and p[0].sentence.id in sents]

611


In [14]:
print len(session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 3).all())

49042


In [111]:
#IGNORE THIS CELL
import numpy as np

splits = set(np.random.choice(np.arange(0,86), 43, replace=False))
dev = set()
test = set()
print len(splits)
with open('small_data/tair_labels.txt', 'rb') as f:
    lines = set(f.readlines())
    print len(lines)
    for i,line in enumerate(lines):
        line=line.strip()
        if i in splits:
            if line in dev: print line
            dev.add(line)
        else:
            if line in test:print line
            test.add(line)
print i
print len(dev), len(test)

[TemporarySpan("", sentence=459557, chars=[62,131], words=[0,0])]
371416
43
86
85
43 43


## @Tara - this is where I'm creating new splits

In [15]:
print sents

for p in session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 4).all():
    print p

set([380931, 423940, 378885, 439627, 410968, 426006, 439641, 460824, 380953, 430107, 374812, 459503, 439644, 403498, 401075, 380995, 374852, 380999, 378955, 443405, 370360, 438357, 456379, 425817, 434290, 374902, 456828, 401534, 456831, 433515, 456837, 401542, 456843, 456848, 403602, 443417, 370372, 403611, 456861, 381086, 456864, 436385, 456049, 403629, 438450, 438454, 405687, 398710, 379447, 439671, 456909, 403662, 456913, 438485, 398714, 377057, 379106, 456933, 414076, 377068, 459475, 419198, 403702, 377096, 456971, 436492, 377103, 436496, 436500, 436502, 411928, 411932, 381216, 438574, 378590, 438582, 438585, 411979, 432480, 429400, 432487, 432495, 381300, 409973, 397688, 381305, 436604, 450943, 450947, 450951, 433219, 444820, 383726, 412056, 444827, 412060, 411035, 430502, 440049, 404892, 414124, 433261, 410034, 432565, 410039, 457157, 436687, 425720, 436691, 457174, 436695, 400938, 436711, 436719, 444917, 436727, 377338, 377341, 410110, 451073, 422998, 455174, 446983, 432654, 444

ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("not secrete nectarTo", sentence=459475, chars=[203,222], words=[28,30]))
ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("accumulate inside the cups formed by sepals", sentence=459475, chars=[386,428], words=[56,62]))
ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("displayed consistent results", sentence=459475, chars=[76,103], words=[10,12]))
ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("surrounding the lateral nectaries", sentence=459475, chars=[430,462], words=[63,66]))
ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("cups formed by sepals surrounding the lateral", sentence=459475, chars=[408,452], words=[59,65]))
ComplexPhenotypes(Span("production", sentence=459475, chars=[298,307], words=[42,42]), Span("individua

ComplexPhenotypes(Span("were viable than WT", sentence=456861, chars=[80,98], words=[10,13]), Span("viable seeds had successfully germinated", sentence=456861, chars=[187,226], words=[32,36]))
ComplexPhenotypes(Span("were viable than WT", sentence=456861, chars=[80,98], words=[10,13]), Span("difference in mean rate of root growth developed after day 3-4 ps", sentence=456861, chars=[111,175], words=[17,28]))
ComplexPhenotypes(Span("were viable than WT", sentence=456861, chars=[80,98], words=[10,13]), Span("fewer fft-1 seeds were viable than WT", sentence=456861, chars=[62,98], words=[7,13]))
ComplexPhenotypes(Span("were viable than WT", sentence=456861, chars=[80,98], words=[10,13]), Span("meant that fewer fft-1 seeds", sentence=456861, chars=[51,78], words=[5,9]))
ComplexPhenotypes(Span("were viable than WT", sentence=456861, chars=[80,98], words=[10,13]), Span("continued to develop", sentence=456861, chars=[233,252], words=[39,41]))
ComplexPhenotypes(Span("were viable than WT", senten

ComplexPhenotypes(Span("expansion", sentence=413020, chars=[106,114], words=[16,16]), Span("lower than in wild-type plants throughout leaf expansion", sentence=413020, chars=[59,114], words=[9,16]))
ComplexPhenotypes(Span("expansion", sentence=413020, chars=[106,114], words=[16,16]), Span("Granule numbers per unit chloroplast volume appeared to", sentence=413020, chars=[0,54], words=[0,7]))
ComplexPhenotypes(Span("lower than in wild-type", sentence=413020, chars=[59,81], words=[9,12]), Span("lower than in wild-type plants throughout leaf expansion", sentence=413020, chars=[59,114], words=[9,16]))
ComplexPhenotypes(Span("lower than in wild-type", sentence=413020, chars=[59,81], words=[9,12]), Span("Granule numbers per unit chloroplast volume appeared to", sentence=413020, chars=[0,54], words=[0,7]))
ComplexPhenotypes(Span("numbers per unit chloroplast volume appeared to", sentence=413020, chars=[8,54], words=[1,7]), Span("lower than in wild-type plants throughout leaf expansion", senten

ComplexPhenotypes(Span("partly due to the enhanced osmotic tolerance mediated by", sentence=417368, chars=[72,127], words=[12,20]), Span("results", sentence=417368, chars=[6,12], words=[1,1]))
ComplexPhenotypes(Span("tolerance observed in", sentence=417368, chars=[38,58], words=[6,8]), Span("partly due to the enhanced osmotic tolerance mediated by NHX1", sentence=417368, chars=[72,132], words=[12,21]))
ComplexPhenotypes(Span("tolerance observed in", sentence=417368, chars=[38,58], words=[6,8]), Span("salt tolerance observed in acs7", sentence=417368, chars=[33,63], words=[5,9]))
ComplexPhenotypes(Span("tolerance observed in", sentence=417368, chars=[38,58], words=[6,8]), Span("results", sentence=417368, chars=[6,12], words=[1,1]))
ComplexPhenotypes(Span("germination", sentence=417602, chars=[52,62], words=[10,10]), Span("hypersensitive to exogenous ABA during germination", sentence=417602, chars=[13,62], words=[5,10]))
ComplexPhenotypes(Span("size and morphology apparent only in some t

...

ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("ssadh mutants as well as the downshift of the F0/Temperature curve to lower temperature", sentence=449549, chars=[59,145], words=[9,22]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("heat-induced rise of F0 fluorescence recorded in ssadh mutants as well as the downshift of", sentence=449549, chars=[10,99], words=[2,16]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("recorded in ssadh mutants as well as the downshift of the F0/Temperature curve to lower", sentence=449549, chars=[47,133], words=[7,21]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("photosystem II is impaired in", sentence=449549, chars=[158,186], words=[25,29]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32

ComplexPhenotypes(Span("showed a statistically significant reduction in the number of", sentence=436492, chars=[147,207], words=[29,37]), Span("Microscopic examination of Hpa colonization revealed that two", sentence=436492, chars=[0,60], words=[0,7]))
ComplexPhenotypes(Span("showed a statistically significant reduction in the number of", sentence=436492, chars=[147,207], words=[29,37]), Span("colonization revealed that two mutants defective in RdDM", sentence=436492, chars=[31,86], words=[4,11]))
ComplexPhenotypes(Span("colonization revealed that two", sentence=436492, chars=[31,60], words=[4,7]), Span("showed a statistically significant reduction in the number of leaves", sentence=436492, chars=[147,214], words=[29,38]))
ComplexPhenotypes(Span("colonization revealed that two", sentence=436492, chars=[31,60], words=[4,7]), Span("Microscopic examination of Hpa colonization revealed that two", sentence=436492, chars=[0,60], words=[0,7]))
ComplexPhenotypes(Span("colonization revealed tha

ComplexPhenotypes(Span("activity", sentence=370288, chars=[257,264], words=[49,49]), Span("primers for GBSS activity", sentence=370288, chars=[240,264], words=[46,49]))
ComplexPhenotypes(Span("activity", sentence=370288, chars=[257,264], words=[49,49]), Span("oligosaccharides", sentence=370288, chars=[205,220], words=[40,40]))
ComplexPhenotypes(Span("mass", sentence=370288, chars=[151,154], words=[29,29]), Span("high amylose content", sentence=370288, chars=[109,128], words=[20,22]))
ComplexPhenotypes(Span("mass", sentence=370288, chars=[151,154], words=[29,29]), Span("mild impairment in the rate of starch degradation", sentence=370288, chars=[31,79], words=[6,13]))
ComplexPhenotypes(Span("mass", sentence=370288, chars=[151,154], words=[29,29]), Span("primers for GBSS activity", sentence=370288, chars=[240,264], words=[46,49]))
ComplexPhenotypes(Span("mass", sentence=370288, chars=[151,154], words=[29,29]), Span("oligosaccharides", sentence=370288, chars=[205,220], words=[40,40]))
Comp

ComplexPhenotypes(Span("was 27% lower compared with the", sentence=383816, chars=[49,79], words=[8,14]), Span("germinated pollen tube length was also affected in", sentence=383816, chars=[89,138], words=[18,25]))
ComplexPhenotypes(Span("was 27% lower compared with the", sentence=383816, chars=[49,79], words=[8,14]), Span("pollen germination rate of vdac1-6 was 27% lower compared with the", sentence=383816, chars=[14,79], words=[3,14]))
ComplexPhenotypes(Span("length was also affected in", sentence=383816, chars=[112,138], words=[21,25]), Span("germinated pollen tube length was also affected in", sentence=383816, chars=[89,138], words=[18,25]))
ComplexPhenotypes(Span("length was also affected in", sentence=383816, chars=[112,138], words=[21,25]), Span("pollen germination rate of vdac1-6 was 27% lower compared with the", sentence=383816, chars=[14,79], words=[3,14]))
ComplexPhenotypes(Span("did not set fertile", sentence=383822, chars=[121,139], words=[26,29]), Span("siliques of the vdac

ComplexPhenotypes(Span("were similar for all concentrations of", sentence=449648, chars=[55,92], words=[10,15]), Span("GHB tested", sentence=449648, chars=[102,111], words=[18,19]))
ComplexPhenotypes(Span("were similar for all concentrations of", sentence=449648, chars=[55,92], words=[10,15]), Span("phenotypes of wild type plants", sentence=449648, chars=[4,33], words=[1,5]))
ComplexPhenotypes(Span("were similar for all concentrations of", sentence=449648, chars=[55,92], words=[10,15]), Span("GHB in the medium", sentence=449648, chars=[211,227], words=[38,41]))
ComplexPhenotypes(Span("growth was correlated with increased amounts of", sentence=449648, chars=[156,202], words=[29,35]), Span("increased inhibition of plant growth was correlated with increased amounts of", sentence=449648, chars=[126,202], words=[25,35]))
ComplexPhenotypes(Span("growth was correlated with increased amounts of", sentence=449648, chars=[156,202], words=[29,35]), Span("GHB tested", sentence=449648, chars=[102,1

In [12]:
import numpy as np

doc_ids = list(set([p[0].sentence.document.id for p in session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 3).all() if p[0].sentence.id in sents]))
print len(doc_ids)

0


In [18]:
np.random.seed(742)
split_size = len(doc_ids)/2
split_1 = np.random.choice(doc_ids, split_size, replace=False)
'''
if session.query(PhenoPair).filter(PhenoPair.split == 1):
    for p in session.query(PhenoPair).filter(PhenoPair.split == 1):
        p.clear()

if session.query(PhenoPair).filter(PhenoPair.split == 2):
    for p in session.query(PhenoPair).filter(PhenoPair.split == 2):
        p.clear()
'''
num_c = 0
for cand in c:
    cand.split = 4
    if cand.get_contexts()[0].sentence.document.id in split_1:
        cand.split = 5
        num_c += 1
session.commit()

print len(c)
print num_c


ValueError: a must be non-empty

In [35]:
for p in session.query(PhenoPairComplex).filter(PhenoPairComplex.split == 4):
    print p

ComplexPhenotypes(Span("overexpression of", sentence=450055, chars=[81,97], words=[19,20]), Span("Salt stress sensitivity of the zip17 T-DNA mutant", sentence=450055, chars=[7,55], words=[5,12]))
ComplexPhenotypes(Span("sensitivity of the", sentence=450055, chars=[19,36], words=[7,9]), Span("Salt stress sensitivity of the zip17 T-DNA mutant", sentence=450055, chars=[7,55], words=[5,12]))
ComplexPhenotypes(Span("early", sentence=438574, chars=[93,97], words=[17,17]), Span("early flowering", sentence=438574, chars=[93,107], words=[17,18]))
ComplexPhenotypes(Span("expression is decreased in the hub1-4", sentence=438574, chars=[17,53], words=[4,9]), Span("early flowering", sentence=438574, chars=[93,107], words=[17,18]))
ComplexPhenotypes(Span("addition", sentence=438574, chars=[3,10], words=[1,1]), Span("early flowering", sentence=438574, chars=[93,107], words=[17,18]))
ComplexPhenotypes(Span("sensitive", sentence=450019, chars=[290,298], words=[52,52]), Span("screened for salt sensitivit

ComplexPhenotypes(Span("germination rate of", sentence=383831, chars=[17,35], words=[3,5]), Span("germinated pollen tubes were also drastically shorter than those of the", sentence=383831, chars=[111,181], words=[17,27]))
ComplexPhenotypes(Span("germination rate of", sentence=383831, chars=[17,35], words=[3,5]), Span("reduced about one-eighth", sentence=383831, chars=[82,105], words=[13,15]))
ComplexPhenotypes(Span("was reduced about", sentence=383831, chars=[78,94], words=[12,14]), Span("germination rate of pollen obtained from vdac2-2", sentence=383831, chars=[17,64], words=[3,9]))
ComplexPhenotypes(Span("was reduced about", sentence=383831, chars=[78,94], words=[12,14]), Span("germinated pollen tubes were also drastically shorter than those of the", sentence=383831, chars=[111,181], words=[17,27]))
ComplexPhenotypes(Span("was reduced about", sentence=383831, chars=[78,94], words=[12,14]), Span("reduced about one-eighth", sentence=383831, chars=[82,105], words=[13,15]))
ComplexPhenot

ComplexPhenotypes(Span("Microscopic examination of", sentence=436492, chars=[0,25], words=[0,2]), Span("colonization revealed that two mutants defective in RdDM", sentence=436492, chars=[31,86], words=[4,11]))
ComplexPhenotypes(Span("Microscopic examination of", sentence=436492, chars=[0,25], words=[0,2]), Span("Microscopic examination of Hpa colonization revealed that two", sentence=436492, chars=[0,60], words=[0,7]))
ComplexPhenotypes(Span("Microscopic examination of", sentence=436492, chars=[0,25], words=[0,2]), Span("showed a statistically significant reduction in the number of leaves", sentence=436492, chars=[147,214], words=[29,38]))
ComplexPhenotypes(Span("defective in", sentence=436492, chars=[70,81], words=[9,10]), Span("colonization revealed that two mutants defective in RdDM", sentence=436492, chars=[31,86], words=[4,11]))
ComplexPhenotypes(Span("defective in", sentence=436492, chars=[70,81], words=[9,10]), Span("Microscopic examination of Hpa colonization revealed that two"

ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("lower heat-induced rise of F0 fluorescence recorded in ssadh mutants as well as the downshift", sentence=449549, chars=[4,96], words=[1,15]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("fluorescence recorded in ssadh mutants as well as the downshift of the F0/Temperature curve to", sentence=449549, chars=[34,127], words=[6,20]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("rise of F0 fluorescence recorded in ssadh mutants as well as the downshift of the", sentence=449549, chars=[23,103], words=[3,17]))
ComplexPhenotypes(Span("possibly due to high", sentence=449549, chars=[201,220], words=[32,35]), Span("F0 fluorescence recorded in ssadh mutants as well as the downshift of the F0/Temperature curve", sentence=449549, chars=[31,124], words=[5,19]))
ComplexPhenotypes(Span("

ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], words=[45,45]), Span("pigment was visible even in young tissues", sentence=456913, chars=[193,233], words=[35,41]))
ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], words=[45,45]), Span("slightly reduced in level compared with the WT", sentence=456913, chars=[125,170], words=[20,27]))
ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], words=[45,45]), Span("sucrose", sentence=456913, chars=[306,312], words=[61,61]))
ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], words=[45,45]), Span("differences in flavonoid content of the plants", sentence=456913, chars=[471,516], words=[87,93]))
ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], words=[45,45]), Span("perturbed in mutant plantsIn fft-1 mutant seedlings up to 1 week of age", sentence=456913, chars=[24,94], words=[3,15]))
ComplexPhenotypes(Span("quantity", sentence=456913, chars=[243,250], wor

...

ComplexPhenotypes(Span("only slightly reduced in", sentence=410168, chars=[55,78], words=[11,14]), Span("number was severely reduced in WT plants", sentence=410168, chars=[9,48], words=[2,8]))
ComplexPhenotypes(Span("number was severely reduced in", sentence=410168, chars=[9,38], words=[2,6]), Span("only slightly reduced in wrky34 plants", sentence=410168, chars=[55,92], words=[11,16]))
ComplexPhenotypes(Span("number was severely reduced in", sentence=410168, chars=[9,38], words=[2,6]), Span("seed number was severely reduced in", sentence=410168, chars=[4,38], words=[1,6]))
ComplexPhenotypes(Span("number was severely reduced in", sentence=410168, chars=[9,38], words=[2,6]), Span("number was severely reduced in WT plants", sentence=410168, chars=[9,48], words=[2,8]))
ComplexPhenotypes(Span("treatment with", sentence=462246, chars=[65,78], words=[12,13]), Span("reduced sizes of the SAMs in clv3-2 seedlings after the treatment with TH-ProCLV3", sentence=462246, chars=[9,89], words=[2,14])

ComplexPhenotypes(Span("black", sentence=443147, chars=[260,264], words=[49,49]), Span("normalized delta plots for APS1/AT3G22890", sentence=443147, chars=[15,55], words=[3,7]))
ComplexPhenotypes(Span("black", sentence=443147, chars=[260,264], words=[49,49]), Span("GRF8/AT4G24150 antisense transcript signals", sentence=443147, chars=[94,136], words=[14,17]))
ComplexPhenotypes(Span("black", sentence=443147, chars=[260,264], words=[49,49]), Span("AP2/AT4G36920", sentence=443147, chars=[75,87], words=[11,11]))
ComplexPhenotypes(Span("black", sentence=443147, chars=[260,264], words=[49,49]), Span("S8", sentence=443147, chars=[256,257], words=[47,47]))
ComplexPhenotypes(Span("20-40% increases in", sentence=443147, chars=[156,174], words=[20,23]), Span("Figures S5", sentence=443147, chars=[233,242], words=[40,41]))
ComplexPhenotypes(Span("20-40% increases in", sentence=443147, chars=[156,174], words=[20,23]), Span("S7", sentence=443147, chars=[249,250], words=[45,45]))
ComplexPhenotypes(Span

ComplexPhenotypes(Span("immature", sentence=412056, chars=[8,15], words=[1,1]), Span("no starch granules were visible by light or", sentence=412056, chars=[137,179], words=[23,30]))
ComplexPhenotypes(Span("immature", sentence=412056, chars=[8,15], words=[1,1]), Span("more starch granules per chloroplast than mature leaves", sentence=412056, chars=[49,103], words=[7,14]))
ComplexPhenotypes(Span("immature", sentence=412056, chars=[8,15], words=[1,1]), Span("immature leaves of wild-type plants", sentence=412056, chars=[8,42], words=[1,5]))
ComplexPhenotypes(Span("immature", sentence=412056, chars=[8,15], words=[1,1]), Span("were visible by light or electron microscopy in immature leaves of ss4 plants", sentence=412056, chars=[156,232], words=[26,38]))
ComplexPhenotypes(Span("consistent with the abnormal germination rates", sentence=456848, chars=[110,155], words=[18,23]), Span("consistent with the abnormal germination rates", sentence=456848, chars=[110,155], words=[18,23]))
ComplexPhenot

ComplexPhenotypes(Span("expression in mature", sentence=438585, chars=[48,67], words=[8,10]), Span("FLC expression in mature siliques", sentence=438585, chars=[44,76], words=[7,11]))
ComplexPhenotypes(Span("3-times lower in", sentence=438585, chars=[81,96], words=[13,15]), Span("FLC expression in mature siliques", sentence=438585, chars=[44,76], words=[7,11]))
ComplexPhenotypes(Span("altered", sentence=438582, chars=[32,38], words=[7,7]), Span("altered seed dormancy phenotype in the flc mutant", sentence=438582, chars=[32,80], words=[7,14]))
ComplexPhenotypes(Span("few more", sentence=421372, chars=[35,42], words=[6,7]), Span("increased rosette leaf number of the spl9 spl15 double mutant", sentence=421372, chars=[169,229], words=[31,40]))
ComplexPhenotypes(Span("few more", sentence=421372, chars=[35,42], words=[6,7]), Span("accounted for by the slight delay in the transition to flowering", sentence=421372, chars=[65,128], words=[12,22]))
ComplexPhenotypes(Span("few more", sentence=4213

ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), Span("WRKY60 in ABA signalling", sentence=411118, chars=[437,460], words=[74,77]))
ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), Span("consistent with the idea that ABI4", sentence=411118, chars=[463,496], words=[79,84]))
ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), Span("ABI5 function downstream of WRKY18", sentence=411118, chars=[398,431], words=[68,72]))
ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), Span("genetic evidence that ABI4", sentence=411118, chars=[367,392], words=[63,66]))
ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), Span("targets of these two WRKY transcription factors", sentence=411118, chars=[515,561], words=[89,95]))
ComplexPhenotypes(Span("transcription", sentence=411118, chars=[541,553], words=[94,94]), 

In [18]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='gold_complex')
sv

<IPython.core.display.Javascript object>

In [19]:
#testing label removal
sv